# Processing

In [ ]:

frequency_downsample = 10
temporal_resolution = 10
output_path = './output'

import os
from JIN_pylib import gjsignal, ProcessUtil
from JIN_pylib import Data3D_XTF
from datetime import timedelta
from tqdm import tqdm
from time import time

if os.path.exists(output_path) is False:
    os.makedirs(output_path)


time_segments = sub_sp.get_chunks(temporal_resolution,is_partial=False)
data_collector = []
data_size = 0
data_io_time = 0
data_process_time = 0

def save_file(data_collector, output_path):
    data = Data3D_XTF.Data3D()
    # make sure all data segments have the same length
    data_length = np.array([d.data.shape[1] for d in data_collector])
    median_length = np.median(data_length)
    for i, d in enumerate(data_collector):
        if d.data.shape[1] < median_length:
            padding = np.zeros((d.data.shape[0], int(median_length - d.data.shape[1])))
            d.data = np.hstack((d.data, padding))
        elif d.data.shape[1] > median_length:
            d.data = d.data[:, :int(median_length)]
    # Now all data segments should have the same length
    data.data = np.array([d.data for d in data_collector])
    data.start_time = data_collector[0].start_time
    data.taxis = np.array([(d.start_time - data.start_time).total_seconds() for d in data_collector])
    data.faxis = data_collector[0].taxis
    data.daxis = data_collector[0].daxis
    output_file = os.path.join(output_path, f'SPEdata_{data.start_time.strftime("%Y%m%d_%H%M%S")}.h5')
    data.saveh5(output_file)
    print(f'Saved data to {output_file}')
    return output_file

for time_seg in tqdm(time_segments):
    bgt, edt = time_seg
    tic = time()
    DASdata = sub_sp.get_data(bgt, edt)
    toc = time()
    data_io_time += toc - tic

    tic = time()

    data = DASdata.data
    N = data.shape[1]// frequency_downsample
    fftdata = 0
    for i in range(frequency_downsample):
        fftdata += np.abs(np.fft.rfft(data[:, i*N:(i+1)*N], axis=1))

    dt = np.median(np.diff(DASdata.taxis)) 
    freq = np.fft.rfftfreq(N, dt)

    SPEdata = DASdata
    SPEdata.data = fftdata
    SPEdata.taxis = freq
    SPEdata.start_time = DASdata.start_time + timedelta(seconds = DASdata.taxis[-1]/2)

    # SPEdata = ProcessUtil.spectrum_transform_2D(DASdata)
    # SPEdata.start_time = DASdata.start_time + timedelta(seconds = DASdata.taxis[-1]/2)
    # N = SPEdata.data.shape[1]
    # N = np.floor(N/frequency_downsample).astype(int)*frequency_downsample
    # SPEdata.data = SPEdata.data[:, :N]
    # SPEdata.data = SPEdata.data.reshape(SPEdata.data.shape[0], -1, frequency_downsample).mean(axis=2)
    # SPEdata.taxis = SPEdata.taxis[:N]
    # SPEdata.taxis = SPEdata.taxis.reshape(-1, frequency_downsample).mean(axis=1)


    toc = time()
    data_process_time += toc - tic

    data_collector.append(SPEdata)
    data_size += SPEdata.data.size

    if data_size > 1e8:
        output_file = save_file(data_collector, output_path)
        data_size = 0
        data_collector = []

if len(data_collector) > 0:
    output_file = save_file(data_collector, output_path)

print(f'Data IO time: {data_io_time:.2f} s')
print(f'Data processing time: {data_process_time:.2f} s')

# FBA processing

In [ ]:
from glob import glob
from JIN_pylib import Data3D_XTF, Data2D_XT
from tqdm import tqdm

files = glob('./output/*.h5')

freq_min = 10
freq_max = 200

FBAdata = Data3D_XTF.get_FBA_from_files(files, freq_min, freq_max)
FBAdata.fill_gap_zeros()
FBAdata.print_info()

# Channel Spectrum